# Verify Segmentation Results

Compare the segmentation results to the ground truths and produce a csv file that lists true positves, true negatives, false positives and false negatives.


In [ ]:
import numpy as np
from skimage import io#, morphology, measure
#from sklearn.cluster import KMeans
from datetime import datetime
import csv
from pathlib import Path
import sys

# add results to a DB
import sqlite3
from sqlite3 import Error

def create_connection(db_file):
    # create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)
    finally:
        if conn:
            return conn
        else:
            conn.close()
            
def create_table(conn, create_table_sql):
    # create a table from the create_table_sql statement
    # :param conn: Connection object
    # :param create_table_sql: a CREATE TABLE statement
    # :return:
    
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

def insert_obj(conn, table, obj):
    obj_keys = ""
    obj_values = ""
    for key in obj:
        if obj_keys =="":
            obj_keys = key
            obj_values = "'" + str(obj[key]) + "'"
        else:
            obj_keys += ", " + key
            obj_values += ", '" + str(obj[key]) + "'"
    insert_stmt = "INSERT INTO {} ({}) VALUES ({})".format(table, obj_keys, obj_values)
    
    cur = conn.cursor()
    cur.execute(insert_stmt)
    conn.commit()
    return cur.lastrowid


# writes data to the given file name
def write_csv_data(values, filename):
    fieldnames = []
    for item in values.keys():
        for key in values[item].keys():
            if not key in fieldnames:
                fieldnames.append(key)
    #write back to a new csv file
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for key in values.keys():
            writer.writerow(values[key])

# get list of unique colour, with a count of the pixels
def get_colours_list(img):
    aimg= np.asarray(img)
    return set( tuple(v) for m2d in aimg for v in m2d )


#get contour pixels from an object
def getcontour(pixels):
  contour=[]
  pixels = set(pixels)
  for pixel in pixels:
    # if any of the eight adjacent pixels is not in the pixel set
    # then the pixel sits in the shape border
    if not (pixel[0]-1,pixel[1]-1) in pixels:
      contour.append(pixel)    
    elif not (pixel[0],pixel[1]-1) in pixels:
      contour.append(pixel)
    elif not (pixel[0]+1,pixel[1]-1) in pixels:
      contour.append(pixel)
    elif not (pixel[0]-1,pixel[1]) in pixels:
      contour.append(pixel)
    elif not (pixel[0]+1,pixel[1]) in pixels:
      contour.append(pixel)
    elif not (pixel[0]-1,pixel[1]+1) in pixels:
      contour.append(pixel)
    elif not (pixel[0],pixel[1]+1) in pixels:
      contour.append(pixel)
    elif not (pixel[0]+1,pixel[1]+1) in pixels:
      contour.append(pixel)
  return contour

# get all pixel coordinates for a given colour
def getobject(img, colour):
    indices = np.where(np.all(np.array(img) == colour, axis=-1))
    pixels = list(zip(indices[0], indices[1]))
    return pixels


# split the indeppedent paths in a large set of nodes
def get_neighbours(point, all_points):
    x = point[0]
    y = point[1]

    other_points = set(all_points)
    
    adjacent_points  = [(x-1,y),(x-1,y-1),(x-1,y+1),(x,y+1),(x,y-1),(x+1,y-1),(x+1,y),(x+1,y+1)]
    neighbours = []
    for pt in adjacent_points:
        if pt in other_points:
            neighbours.append(pt)
    return neighbours

# get the points which are 
def get_extremes(path_set, all_points):
    extremes = []
    path_set = set(path_set)
    for point in path_set:
        point_neighbours = set(get_neighbours(point, all_points))
        diff_set = list(point_neighbours.difference(path_set))
        if len(diff_set) != 0:
            extremes.append(point)
    return extremes

# starting from a point in the borders get all which are together
def get_path(start, big_set):
    path_set = []
    neighbours = get_neighbours(start, big_set)
    path_set = [*neighbours]
    path_set.append(start)

    while True:
        extremes  = get_extremes(path_set, big_set)
        if extremes == []:
            break
        for extreme in extremes:
            start = extreme
            neighbours = get_neighbours(start, big_set)
            path_set = [*path_set, *list(set(neighbours).difference(set(path_set)))]
    return path_set

# get the corners of the circunscribed rectangle
def getcontourcorners(shape_contour):
    min_x = min_y = max_x = max_y = 0
    for pair in shape_contour:
        if min_x==min_y==max_y==max_x==0:
            min_y, min_x = pair
            max_y, max_x = pair
        else:
            if pair[0] < min_y:
                min_y = pair[0]
            if pair[1] < min_x:
                min_x = pair[1]
            if pair[0] > max_y:
                max_y = pair[0]
            if pair[1] > max_x:
                max_x = pair[1]
    mid_y = int(round(min_y + (max_y-min_y)/2))
    mid_x = int(round(min_x + (max_x-min_x)/2))
    return ((min_y, min_x), (max_y, max_x))

# get the centre pixel for the circunscribed object
def getcontourcentre(shape_contour):
    min_x = min_y = max_x = max_y = 0
    ((min_y, min_x), (max_y, max_x)) = getcontourcorners(shape_contour)
    mid_y = int(round(min_y + (max_y-min_y)/2))
    mid_x = int(round(min_x + (max_x-min_x)/2))
    return (mid_y, mid_x)

# verify if the path is a hole,
# i.e. contained inside another path
def is_hole(single_path, all_paths):
    for one_path in all_paths:
        if set(single_path) != set(one_path ):
            if len(single_path) < len(one_path):
                ((e,f),(g,h)) = getcontourcorners(single_path)
                ((a,b),(c,d)) = getcontourcorners(one_path)
                if e >= a and e <= c and \
                   f >= b and f <= d and \
                   g >= a and g <= c and \
                   h >= b and h <= d:
                    return True
    return False

def contourcolours(shape_contour, lbl_img):
    contour_colours = {}
    for point in shape_contour:
        point_colour = tuple(lbl_img[point])
        if point_colour in contour_colours.keys():
            contour_colours[point_colour] += 1
        else:
            contour_colours[point_colour] = 1
    return contour_colours

# use the colours on the borders to determine
# an objects class, the proportion
def assignclass(shape_contour, lbl_img):
    contour_colours = contourcolours(shape_contour, lbl_img)
    class_colour = tuple()
    max = 0
    for contour_colour in contour_colours:
        if max == 0:
            max = contour_colours[contour_colour]
            class_colour = contour_colour
        elif max < contour_colours[contour_colour]:
            max = contour_colours[contour_colour]
            class_colour = contour_colour
            
    return class_colour
        

# get all closed independent paths in a set of points
def get_all_paths(big_set):
    not_inspected = big_set
    all_paths = []
    while not_inspected  != []:
        path_set = get_path(not_inspected[0], not_inspected)
        not_inspected = [*list(set(not_inspected).difference(set(path_set)))]
        all_paths.append(path_set)
    # need to remove wholes which are also detected as paths            
    holes=[]
    for single_path in all_paths:
        if is_hole(single_path, all_paths):
            holes.append(single_path)
    if len(holes) != 0:
        for a_hole in holes:
            all_paths.remove(a_hole)
    return all_paths


#***************************************
# A-B) open predictions file and
#      object get borders
# Returns:
#      Dictionary with colours as keys
#      and shape borders lists as
#      elements
#        {(colour): [(paths),...]}
#***************************************
def get_shapes_per_colour(filename):
    img = io.imread(filename)

    # get image size and number of pixel elements
    print("gspc start:  ",datetime.now().strftime("%Y/%m/%d %H:%M:%S"))
    rows, cols, bands = img.shape
    # discard alpha channel
    if bands > 3:
        img = img[:,:,:3]
        bands = 3
    # get a list of colour with pixel count
    print("gspc get colours list 1:  ",datetime.now().strftime("%Y/%m/%d %H:%M:%S"))
    colours_list = get_colours_list(img)
    print("gspc get colours list 2:  ",datetime.now().strftime("%Y/%m/%d %H:%M:%S"))
    # get number of unique colours
    num_clusters = len(colours_list)
    # identify background colour
    backgrounds_skip = [(0, 0, 0)]
    background_clr = tuple(img[0,0])
    if not background_clr in backgrounds_skip:
        backgrounds_skip.append(background_clr)
    #get all shapes for each colour
    objects = {}
    print("gspc process colours list 1:  ",datetime.now().strftime("%Y/%m/%d %H:%M:%S"))
    for this_colour in colours_list:
        if not this_colour in backgrounds_skip:
            print("gspc get objects 1:  ",datetime.now().strftime("%Y/%m/%d %H:%M:%S"))
            object_pixels = getobject(img, this_colour)
            print("gspc get objects 2:  ",datetime.now().strftime("%Y/%m/%d %H:%M:%S"))
            contour_pixels = getcontour(object_pixels)
            all_paths  = get_all_paths(contour_pixels)
            objects[this_colour] = all_paths
    print("gspc process colours list 2:  ",datetime.now().strftime("%Y/%m/%d %H:%M:%S"))
    return objects

# retrieve test files using tr/ts proportions
def test_files_list(source_dir,tr=8,ts=2):
    i_counter = 0
    files_list = []
    for filepath in sorted(source_dir.glob('*.JPG')):
        i_counter += 1
        if i_counter > tr:
            files_list.append(Path(filepath).name[:-4])
            if i_counter == tr+ts:
                i_counter = 0
    return files_list

# ground truth dataset
def get_gt_values(argv,label_colors=None):
    print("Start",datetime.now().strftime("%Y/%m/%d %H:%M:%S"), 'Arguments:', argv)
    try:
        gt_path = argv[0]
        pr_path = argv[1]
        out_file = argv[2]
        train_prop = int(argv[3])
        test_prop = int(argv[4])
    except:
        print("provide five arguments:"+
              "\n -string ground truths path\n -string predictions path"+
              "\n -string output filename (csv)\n -integer proportion train set"+
              "\n -integer proportion test/eval set")
        return
    # insert objects directly into SQLite DB
    # set table name
    table_name = out_file[:-4]
    # create DB
    conn = create_connection(r"gtprverify.sqlite")
    # create table
    create_stmt = " CREATE TABLE IF NOT EXISTS "+ table_name + \
                  "(id integer PRIMARY KEY, file text, " + \
                  "source text, obj_colour text, ground_truth text, "+ \
                  "predicted text); "
    create_table(conn, create_stmt)

    
    gt_dir= Path(gt_path)
    # predictions dataset
    pr_dir = Path(pr_path)
    # paths for ground truth set on semseg directory
    images_dir = gt_dir / 'images'
    labels_dir = gt_dir / 'labels'
    instances_dir = gt_dir / 'instances'

    # get a list of the files to compare
    file_list = test_files_list(images_dir,train_prop,test_prop)

    print("Start",datetime.now().strftime("%Y/%m/%d %H:%M:%S"))
    print(file_list)

    for filename in file_list:
        # GT labels file
        gt_lbl = Path(labels_dir, filename+'_labels.png')
        # GT instances file
        gt_ins = Path(instances_dir, filename+'_instances.png')
        # predictions labels file
        pr_lbl = Path(pr_dir, filename+'_labels.png')
        # predictions instances file
        pr_ins = Path(pr_dir, filename+'_instances.png')

        pr_lbl_img = io.imread(pr_lbl)
        # get image size and number of pixel elements
        rows, cols, bands = pr_lbl_img.shape
        # discard alpha channel
        if bands > 3:
            pr_lbl_img = pr_lbl_img[:,:,:3]
            bands = 3
        gt_lbl_img = io.imread(gt_lbl)
        # get image size and number of pixel elements
        rows, cols, bands = gt_lbl_img.shape
        # discard alpha channel
        if bands > 3:
            gt_lbl_img = gt_lbl_img[:,:,:3]
            bands = 3

        #***************************************************
        # Calculate TP, TN, FP, and FN for earch prediction
        #***************************************************
        # e) open the GT instance file
        # f) for each colour in the instance, get borders
        gt_objects = get_shapes_per_colour(gt_ins)

        for an_object in gt_objects:
            # g) get the types assigned to each object in GT
            # h) comparte to  PR labels to get FN
            for fragment in gt_objects[an_object]:
                f_centre = getcontourcentre(fragment)
                pr_class = tuple(pr_lbl_img[f_centre])
                gt_class = tuple(gt_lbl_img[f_centre])
                ob_values = {"file":filename, "source":"GT","obj_colour":an_object, "ground_truth":gt_class, "predicted":pr_class}
                new_id = insert_obj(conn, table_name, ob_values)
                print("inserted object with id:", new_id, ob_values)
            
        # a) open the predictions instance file
        # b) for each colour in the instance, get borders.
        pr_objects = get_shapes_per_colour(pr_ins)

        for an_object in pr_objects:
            # c) get the types assigned to each object fragment in predictions and GT
            # d) compare to GT labels  to get TP and FP
            for fragment in pr_objects[an_object]:
                f_centre = getcontourcentre(fragment)
                # correct for difference in size of images
                if f_centre[0] >= rows:
                    f_centre = (-1, f_centre[1])
                if f_centre[1] >= cols:
                    f_centre = (f_centre[0], cols -1)    
                assign_pr_class = assignclass(fragment, pr_lbl_img)
                gt_class = tuple(gt_lbl_img[f_centre])
                ob_values = {"file":filename, "source":"PR", "obj_colour":an_object, "ground_truth":gt_class, "predicted":assign_pr_class}
                new_id = insert_obj(conn, table_name, ob_values)
                print("inserted object with id:", new_id, ob_values)
        
    # i) categorise results to obtain TP, TN, FP and FN for each file
    # Add to DB, use queries to summarise
    print("finished:", datetime.now().strftime("%Y/%m/%d %H:%M:%S"))        
    conn.close()


In [ ]:
root_path = Path("E:\\semseg\\semsegdata\\")
# GT labels file
labels_dir = 'pub_data\\microscopy_slides\\nhmmsi'
instances_dir = 'pub_data\\microscopy_slides\\nhmmsi'
pr_dir = 'pred_ms_02\\model_nhm_data_nhm08'
filename = '010646759_816445_1431072'
gt_lbl = root_path / Path(labels_dir, filename+'_labels.png')
# GT instances file
gt_ins = root_path / Path(instances_dir, filename+'_instances.png')
# predictions labels file
pr_lbl = root_path / Path(pr_dir, filename+'_labels.png')
# predictions instances file
pr_ins = root_path / Path(pr_dir, filename+'_instances.png')
print(gt_lbl,"\n",gt_ins,"\n",pr_lbl, "\n",pr_ins)

pr_lbl_img = io.imread(pr_lbl)
# get image size and number of pixel elements #345
rows, cols, bands = pr_lbl_img.shape
# discard alpha channel
if bands > 3:
    pr_lbl_img = pr_lbl_img[:,:,:3]
    bands = 3
gt_lbl_img = io.imread(gt_lbl)
# get image size and number of pixel elements
rows, cols, bands = gt_lbl_img.shape
# discard alpha channel
if bands > 3:
    gt_lbl_img = gt_lbl_img[:,:,:3]
    bands = 3
print("Start:  ",datetime.now().strftime("%Y/%m/%d %H:%M:%S"))
#gt_objects = get_shapes_per_colour(gt_ins)
#len(gt_objects)
#gt_objects.keys()
print("Finish: ",datetime.now().strftime("%Y/%m/%d %H:%M:%S"))

In [ ]:
import cv2

def show_image(img_arr, title = "test"):
    cv2.imshow(title, img_arr)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

show_image(pr_lbl_img, "PR labels")
show_image(gt_lbl_img, "GT labels")

im = cv2.imread(str(pr_ins))

show_image(im, "gt instances")
colours_list = get_colours_list(im)
backgrounds_skip = [(0, 0, 0)]
background_clr = tuple(im[0,0])
if not background_clr in backgrounds_skip:
  backgrounds_skip.append(background_clr)

img_contours = {}
for this_colour in colours_list:
    if not this_colour in backgrounds_skip:
        test = im.copy()
        for mk_black in colours_list:
            if mk_black != this_colour:
                test[np.all(test == mk_black, axis=-1)] = (0,0,0)
        test[np.all(test == this_colour, axis=-1)] = (255,255,255)
        show_image(test, "img copy blob")
        imgray = cv2.cvtColor(test,cv2.COLOR_BGR2GRAY)
        ret,thresh = cv2.threshold(imgray,127,255,cv2.THRESH_BINARY)
        contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        #print(image)
        #print(contours)
        img_contours[this_colour]=contours
        print("Colour:", this_colour, "Contours:", len(contours))
test = im.copy()
for this_colour in colours_list:
    if this_colour in backgrounds_skip:
        test[np.all(test == this_colour, axis=-1)] = (0,0,0)
    else :
        test[np.all(test == this_colour, axis=-1)] = (255,255,255)

show_image(test, "img copy bw")

for col_key in img_contours:
    #print(img_contours[col_key])   
    s_color = tuple ([int(x) for x in col_key])
    test = cv2.drawContours(test, img_contours[col_key], -1, s_color,1)
    for a_point in img_contours[col_key]:
        print(len(a_point), col_key)
        for i_point in a_point:
            test[i_point[0][1],i_point[0][0]] = s_color

show_image(test, "img copy contours")


In [ ]:
color_fragments = {}
for col_key in img_contours:
    #print(img_contours[col_key])   
    for a_point in img_contours[col_key]:
        #print(len(a_point), a_point[0][0], col_key)
        test_point=(a_point[0][0][1],a_point[0][0][0])
        #print(test_point)
        pr_class = tuple(pr_lbl_img[test_point])
        gt_class = tuple(gt_lbl_img[test_point])
        if not col_key in color_fragments:
            color_fragments[col_key] = [{'fragments':1,'PR':pr_class,"GT":gt_class}]
        else:
            for col_set in color_fragments[col_key]:
                if col_set['PR'] == pr_class and col_set['GT'] == gt_class:
                    col_set['fragments'] +=1
                elif gt_class != (0, 0, 0):
                    color_fragments[col_key].append({'fragments':1,'PR':pr_class,"GT":gt_class})
        print(col_key, " CLASSES PR:", pr_class, "CLASSES GT:", gt_class)
print(color_fragments)
len(color_fragments)

In [ ]:
pr2_lbl_img = cv2.imread(str(pr_lbl))
# get image size and number of pixel elements #345
gt2_lbl_img = cv2.imread(str(gt_lbl))
# get image size and number of pixel elements



show_image(pr2_lbl_img, "PR labels")
show_image(gt2_lbl_img, "GT labels")


for col_key in img_contours:
    #print(img_contours[col_key])   
    cnt = img_contours[col_key]
    M = cv2.moments(cnt[0])
    #print(M)
    cx = cy = 0
    if M['m00'] != 0:
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])
    #print(cx,cy)
    test_point = (cy,cx)
    pr_class = tuple(pr2_lbl_img[test_point])
    gr_class = tuple(gt2_lbl_img[test_point])
#    for a_point in img_contours[col_key]:
#        #print(len(a_point), a_point[0][0], col_key)
#        test_point=(a_point[0][0][1],a_point[0][0][0])
#        #print(test_point)
#        pr_class = tuple(pr2_lbl_img[test_point])
#        gr_class = tuple(gt2_lbl_img[test_point])                      
    print(col_key, "CLASSES PR:", pr_class, "CLASSES GT:", gt_class)


In [ ]:
tuple(pr_lbl_img[75,415])

In [ ]:
# using CV2
import cv2
im = None
def show_image(img_arr, title = "test"):
    cv2.imshow(title, img_arr)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def get_img_contours(im):
    #show_image(im, "gt instances")
    colours_list = get_colours_list(im)
    backgrounds_skip = [(0, 0, 0)]
    background_clr = tuple(im[0,0])
    if not background_clr in backgrounds_skip:
      backgrounds_skip.append(background_clr)

    img_contours = {}
    for this_colour in colours_list:
        if not this_colour in backgrounds_skip:
            test = im.copy()
            for mk_black in colours_list:
                if mk_black != this_colour:
                    test[np.all(test == mk_black, axis=-1)] = (0,0,0)
            test[np.all(test == this_colour, axis=-1)] = (255,255,255)
            #show_image(test, "img copy blob")
            imgray = cv2.cvtColor(test,cv2.COLOR_BGR2GRAY)
            ret,thresh = cv2.threshold(imgray,127,255,cv2.THRESH_BINARY)
            contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
            #print(image)
            #print(contours)
            img_contours[this_colour]=contours
    return img_contours

def get_cnt_centre(cnt):
    #print("Contour:", cnt)
    M = cv2.moments(cnt)
    c_ctr = None
    if M['m00']!=0:
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])
        c_ctr = (cy,cx)
    else:
        c_ctr = get_cnt_centre2(cnt)
    return c_ctr

# get the corners of the circunscribed rectangle
def get_cnt_corners(shape_contour):
    #print(shape_contour)
    min_x = min_y = max_x = max_y = 0
    for pair in shape_contour:
        #print("Pair:", pair)
        if min_x==min_y==max_y==max_x==0:
            min_y, min_x = pair[0]
            max_y, max_x = pair[0]
        else:
            if pair[0][0] < min_y:
                min_y = pair[0][0]
            if pair[0][1] < min_x:
                min_x = pair[0][1]
            if pair[0][0] > max_y:
                max_y = pair[0][0]
            if pair[0][1] > max_x:
                max_x = pair[0][1]
    #print ((min_y, min_x), (max_y, max_x))
    return ((min_y, min_x), (max_y, max_x))

# get the centre pixel for the circunscribed object
def get_cnt_centre2(shape_contour):
    #print(shape_contour)
    min_x = min_y = max_x = max_y = 0
    ((min_y, min_x), (max_y, max_x)) = get_cnt_corners(shape_contour)
    mid_y = int(round(min_y + (max_y-min_y)/2))
    mid_x = int(round(min_x + (max_x-min_x)/2))
    return (mid_y, mid_x)
    
def get_pr_vals(file_list = ['010646729_816445_1431072', '010646739_816445_1431072']):
    root_path = Path().absolute()
    # GT labels file
    labels_dir = 'data/raw/labels'
    instances_dir = 'data/raw/instances'
    pr_dir = 'predictions/model_nhm_data_nhm'
    print("Start:  ",datetime.now().strftime("%Y/%m/%d %H:%M:%S"))
    for filename in file_list:
        gt_lbl = root_path / Path(labels_dir, filename+'_labels.png')
        # GT instances file
        gt_ins = root_path / Path(instances_dir, filename+'_instances.png')
        # predictions labels file
        pr_lbl = root_path / Path(pr_dir, filename+'_labels.png')
        # predictions instances file
        pr_ins = root_path / Path(pr_dir, filename+'_instances.png')
        #print(gt_lbl,"\n",gt_ins,"\n",pr_lbl, "\n",pr_ins)
        pr_lbl_img = cv2.imread(str(pr_lbl))
        #show_image(pr_lbl_img, "PR labels")
        gt_lbl_img = cv2.imread(str(gt_lbl))
        new_id = 1
        #show_image(gt_lbl_img, "GT labels")
        #***************************************************
        # Calculate TP, TN, FP, and FN for earch prediction
        #***************************************************
        # a) open the GT instance file
        # b) for each colour in the instance, get borders
        gt_ins_img = cv2.imread(str(gt_ins))
        gt_objects = get_img_contours(gt_ins_img)

        # c) open the predictions instance file
        # d) for each colour in the instance, get borders.
        pr_ins_img = cv2.imread(str(pr_ins))
        pr_objects = get_img_contours(pr_ins_img)
        # c) get the types assigned to each object fragment in predictions and GT
        # d) compare to GT labels  to get TP and FP
        for object_frags in pr_objects:
            #print(object_frags, "FRAGEMENTS", len(pr_objects[object_frags]))
            for fragment in pr_objects[object_frags]:   
                f_centre = get_cnt_centre(fragment)
                if f_centre[0] >= rows:
                    f_centre = (-1, f_centre[1])
                if f_centre[1] >= cols:
                    f_centre = (f_centre[0], cols -1)    
                gt_class = tuple(gt_lbl_img[f_centre])
                pr_class = tuple(pr_lbl_img[f_centre])
                ob_values = {"file":filename, "source":"PR", "obj_colour":object_frags, "ground_truth":gt_class, "predicted":pr_class}
                #new_id = insert_obj(conn, table_name, ob_values)
                print("inserted object:", ob_values, "with id:", new_id)  
                new_id += 1
    print("Finish: ",datetime.now().strftime("%Y/%m/%d %H:%M:%S"))



In [ ]:
files = ['010646729_816445_1431072', '010646739_816445_1431072', '010646749_816445_1431072']
files = ['010646729_816445_1431072', '010646739_816445_1431072', '010646749_816445_1431072', '010646759_816445_1431072', '010646782_816445_1431072',
         '010646804_816445_1431072', '010647287_133431_1431072', '010647327_133431_1431072', '010647349_133431_1431072', '010647370_133431_1431072',
         '010648240_133432_1431072', '010648265_816385_1428365', '010648285_816385_1428365', '010648310_816385_1428371', '010649246_816387_1430143',
         '010649755_816388_1427961', '010649779_816388_1427961', '010649804_816388_1427965', '010649837_816388_1427970', '010651082_816390_1428049',
         '010651108_816390_1428051', '010651146_816390_1428072', '010651166_816390_1428058', '010652955_816393_1428654', '010652984_816393_1428654',
         '010653012_816393_1428658', '010653030_816393_1428659', '010654762_816395_1428700', '010654794_816395_1428700', '010654832_816395_1428700',
         '010656162_816398_1429274', '010656188_816398_1432062', '010656205_816398_1429260', '010656239_816398_1429280', '010657813_816401_1430130',
         '010657836_816401_1430130', '010657866_816401_1430112', '010659072_133420_1430424', '010661101_816405_1430633', '010663572_816408_1431362',
         '010665274_108803_1431064', '010668463_816414_1428267', '010671454_816417_1428585', '010671729_816419_1428737', '010671908_816419_1430765',
         '010671941_816419_1430762', '010672669_816420_1428706', '010672896_816418_1428597', '010673160_816418_1428595', '010692135_816271_1432257']
files = ['010646759_816445_1431072']
#files = ['010646729_816445_1431072']
get_pr_vals(files)

In [ ]:
# get the corners of the circunscribed rectangle
def get_cnt_corners(shape_contour):
    print(shape_contour)
    min_x = min_y = max_x = max_y = 0
    for pair in shape_contour:
        if min_x==min_y==max_y==max_x==0:
            min_y, min_x = pair[0]
            max_y, max_x = pair[0]
        else:
            if pair[0][0] < min_y:
                min_y = pair[0][0]
            if pair[0][1] < min_x:
                min_x = pair[0][1]
            if pair[0][0] > max_y:
                max_y = pair[0][0]
            if pair[0][1] > max_x:
                max_x = pair[0][1]
    #print ((min_y, min_x), (max_y, max_x))
    return ((min_y, min_x), (max_y, max_x))

# get the centre pixel for the circunscribed object
def get_cnt_centre(shape_contour):
    print(shape_contour)
    min_x = min_y = max_x = max_y = 0
    ((min_y, min_x), (max_y, max_x)) = get_cnt_corners(shape_contour)
    mid_y = int(round(min_y + (max_y-min_y)/2))
    mid_x = int(round(min_x + (max_x-min_x)/2))
    return (mid_y, mid_x)


x = [[[540, 210]], [[540, 221]]] 
((min_y, min_x), (max_y, max_x)) = get_cnt_corners(x)
print((min_y, min_x), (max_y, max_x))
c_ctr =  get_cnt_centre(x)
print(c_ctr)

In [ ]:
# get the points which are 
def get_extremes(path_set, all_points):
    extremes = []
    path_set = set(path_set)
    for point in path_set:
        point_neighbours = set(get_neighbours(point, all_points))
        diff_set = list(point_neighbours.difference(path_set))
        if len(diff_set) != 0:
            extremes.append(point)
    return extremes
